In [1]:
%matplotlib inline

In [2]:
from itertools import product
from os.path import *
from os import *
import cPickle as pickle
import sys

In [3]:
import matplotlib as mpl
import matplotlib.pyplot as plt

In [4]:
from icecube.dataclasses import *
from icecube import *
from icecube.icetray import *

In [5]:
from icecube.clsim import MakeIceCubeMediumProperties

In [6]:
from plotGoodies import removeBorder

In [7]:
!pwd

/home/justin/src/retro/notebooks


In [8]:
sys.path.append(expandvars('$I3_SRC/clsim/python/util'))
sys.path.append(abspath('..'))

from GetIceTiltZShift import GetIceTiltZShift, I3CLSimScalarFieldIceTiltZShift
from retro.const import get_sd_idx
from retro.utils.geom import powerspace

In [10]:
gcd = pickle.load(file('../data/GeoCalibDetectorStatus_IC86.55697_corrected_V2.pkl', 'rb'))
geo = gcd['geo']

See `ice_layer_properties_validation.ipynb` for validation that I'm computing layer indices using the CLSim tool correctly.

In [19]:
ice_model = 'spice_lea'

In [20]:
ice_props, ice_params = MakeIceCubeMediumProperties(
    detectorCenterDepth=1948.07*I3Units.m,
    iceDataDirectory=expandvars('$I3_SRC/ice-models/resources/models/' + ice_model),
    useTiltIfAvailable=True,
    returnParameters=True
)
ice_params

{'anisotropyDirAzimuth': 3.7699111843077517,
 'anisotropyMagnitudeAlongDir': 0.04,
 'anisotropyMagnitudePerpToDir': -0.08}

In [21]:
!cat /cvmfs/icecube.opensciencegrid.org/py2-v3/Ubuntu_16.04_x86_64/metaprojects/combo/stable/ice-models/resources/models/$ice_model/cfg.txt

# ppc configuration file: follow strict order below
5     # over-R: DOM radius "oversize" scaling factor
1.0   # overall DOM efficiency correction
0.41  # 0=HG; 1=SAM
0.9   # g=<cos(theta)>

216   # direction of ice tilt (perp. to flow)
0.04  # magnitude of ice anisotropy along tilt
-0.08 # magnitude of ice anisotropy along flow


In [22]:
rootdir = (
    '/cvmfs/icecube.opensciencegrid.org/py2-v3/Ubuntu_16.04_x86_64/'
    'metaprojects/combo/stable/ice-models/resources/models/{}'
).format(ice_model)

cfg = np.loadtxt(join(rootdir, 'cfg.txt'))
print 'cfg.txt shape:', cfg.shape

oversize_scaling = cfg[0]
efficiency_correction = cfg[1]
liu_scattering_fraction = cfg[2]
mean_cos_scattering_theta = cfg[3]

print 'mean_cos_scattering_theta:', mean_cos_scattering_theta

icemodel_par = np.loadtxt(join(rootdir, 'icemodel.par'))
print 'icemodel.par shape:', icemodel_par.shape
if False: #ice_model == 'spice_mie':
    assert icemodel_par.shape[0] == 6
    alpha, kappa, A, B, D, E = icemodel_par[:, 0].T
    print 'alpha={}, kappa={}, A={}, B={}, D={}, E={}'.format(alpha, kappa, A, B, D, E)
else: # ice_model == 'spice_lea':
    #assert icemodel_par.shape[0] == 4
    alpha, kappa, A, B = icemodel_par[:, 0].T[:4]
    print 'alpha={}, kappa={}, A={}, B={}'.format(alpha, kappa, A, B)
#else:
#    raise ValueError(ice_model)
    
icemodel = np.loadtxt(join(rootdir, 'icemodel.dat'))
print 'icemodel.dat shape:', icemodel.shape

# first column: z layer *depth* midpoints
zlayermidpts = icemodel[:, 0]
# second column: b_e(400) values
be400 = icemodel[:, 1]
# third column: adust(400) values
adust400 = icemodel[:, 2]
# fourth column: delta_tau values
delta_tau = icemodel[:, 3]

cfg.txt shape: (7,)
mean_cos_scattering_theta: 0.9
icemodel.par shape: (4, 2)
alpha=0.898608505726, kappa=1.08410680294, A=6954.09033203, B=6617.75439453
icemodel.dat shape: (171, 4)


In [23]:
%%time

wavelen = 555e-9 * I3Units.m
n_strings = 86
n_doms_per_string = 60
n_doms = 86 * 60
z0 = ice_props.GetLayersZStart()
dz = ice_props.GetLayersHeight()
zshift = ice_props.GetIceTiltZShift()

properties = [None]*n_doms
for s_idx, d_idx in product(range(n_strings), range(n_doms_per_string)):
    string = s_idx + 1
    dom = d_idx + 1
    sd_idx = get_sd_idx(string=string, dom=dom)
    x, y, z = geo[s_idx, d_idx]
    sl = []
    al = []
    for r, costheta, phi in product(
            powerspace(0.1, 200, 5, 5),
            np.linspace(np.cos(np.pi), np.cos(0), 9),
            np.arange(0, 2*np.pi, np.pi/4)
        ):
        sintheta = np.sin(np.arccos(costheta))
        z_ = z + r * costheta
        x_ = x + r * sintheta * np.cos(phi)
        y_ = y + r * sintheta * np.sin(phi)

        # Original sign convention: z + zshift - z0; this has now been
        # verified by comparing against a "manual" calculation of the
        # ice scattering length.
        layer_num = int((z_ - z0 + zshift.GetValue(x_, y_, z_)) / dz)
        # Alternative sign convention
        layer_num = int((z_ - z0 - zshift.GetValue(x_, y_, z_)) / dz)

        al.append(ice_props.GetAbsorptionLength(layer_num).GetValue(wavelen))
        sl.append(ice_props.GetScatteringLength(layer_num).GetValue(wavelen))

    zequiv = z + zshift.GetValue(x, y, z)
    layer_num = int((zequiv - z0) / dz)
    properties[sd_idx] = (
        [
            x,
            y,
            z,
            zequiv,
            get_hc0_coord(x, y),
            get_hc1_coord(x, y),
            get_hc2_coord(x, y),
            ice_props.GetAbsorptionLength(layer_num).GetValue(wavelen),
            ice_props.GetScatteringLength(layer_num).GetValue(wavelen)
        ]
        + al
        + sl
    )
properties = np.array(properties)

CPU times: user 22.2 s, sys: 360 ms, total: 22.6 s
Wall time: 22.3 s


In [24]:
np.save('../data/dom_clustering_properties_binned.npy', properties)